In [68]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import re
import json
import time
import datetime

# An Analysis of Movie Performance


In this part, you’ll gather data about popular movies and award winners. The goal is to build a dataset that you’ll later use to analyze what makes a movie successful and how awards and box office performance relate to one another.

### Part 1: Data Gathering

#### 1. Scrape Best Picture Data.  
    * Scrape the [Best Picture wikipedia page](https://en.wikipedia.org/wiki/Academy_Award_for_Best_Picture).  
    * Extract for each movie:  
        * Year  
        * Film Title  
        * Winner (Yes/No)  
    * Data cleaning tips:  
        * Ensure that year and film title columns are clean and consistent (no footnotes, parentheses, etc.).
        * Save the results as best_picture.csv.  

#### Scraping site to get soup

In [7]:
endpoint = 'https://en.wikipedia.org/wiki/Academy_Award_for_Best_Picture'

headers = {
    'User-Agent': 'Movie_Agent'
}


response = requests.get(endpoint, headers = headers)
response

<Response [200]>

In [8]:
soup = BeautifulSoup(response.text)
#print(soup.prettify())

In [9]:
movies = soup.findAll('tr', attrs={'style' : 'background:#FAEB86'}) #searching page for tr row of table that is styled in yellow #FAEB86
movies[0]

<tr style="background:#FAEB86">
<th rowspan="3"><a href="/wiki/1928_in_film" title="1928 in film">1927/28</a><br/><span style="font-size: 85%;"><a href="/wiki/1st_Academy_Awards" title="1st Academy Awards">(1st)</a></span>
</th>
<td><i><b><a href="/wiki/Wings_(1927_film)" title="Wings (1927 film)">Wings</a></b></i>
</td>
<td><b><a href="/wiki/Famous_Players%E2%80%93Lasky" title="Famous Players–Lasky">Famous Players–Lasky</a> <span style="font-size: 85%;">(<a href="/wiki/Lucien_Hubbard" title="Lucien Hubbard">Lucien Hubbard</a>, <a href="/wiki/Jesse_L._Lasky" title="Jesse L. Lasky">Jesse L. Lasky</a>, <a href="/wiki/B._P._Schulberg" title="B. P. Schulberg">B. P. Schulberg</a>, &amp; <a href="/wiki/Adolph_Zukor" title="Adolph Zukor">Adolph Zukor</a>, producers)</span></b>
</td></tr>

In [10]:
movies[0].td.text.strip() #experiementing with how to drill down to the title alone

'Wings'

In [11]:
winning_titles = [movie.td.text.strip() for movie in movies] #iterating over the list of tags to locate the first td tag and extract the movie title
#winners[0]

In [12]:
# Get all wikitables
all_tables = soup.findAll('table', attrs={'class' : 'wikitable'})
# Find just the tables with movie data by filtering on the 'Year of Film Release' column header
# This excludes the 2 wikitables at the very bottom of the page ('Age superlatives' and 'Production companies and distributors with multiple nominations and wins')
movie_tables = [table for table in all_tables if 'Year of Film Release' in table.find('tr').text]

In [13]:
# Create empty list to store dictionaries of movie data, ex. [{'Title': 'Wings', 'Movie_Year': '1927', 'Awards_Year': '1928', 'Winner':'True'}]
movie_info = []
# Iterate through all tables to extract movie data
for table in movie_tables:
    # Find all 'tr' tags ('table row'), skipping the first one since it just contains column headers
    rows = table.findAll('tr')[1:]
    # Iterate through all rows of the table to find year, movie, and winner status
    for row in rows:
        # If the row contains a 'th' ('table header') tag, extract the year and store it in a variable
        if len(row.findAll('th')) != 0:
            awards_year = row.th.a.text
            # If the awards_year contains a slash, only grab the later year
            if '/' in awards_year:
                awards_year = awards_year[:2] + awards_year[-2:]
        # Get the movie title in this row, if there is one
        if len(row.findAll('td')) != 0:
            title = row.td.text.strip()
            # Get the winner status by seeing if the title is in the winning_titles list
            if title in winning_titles:
                winner='Yes'
            else:
                winner='No'
        else:
            title = ''
        # If this row has a movie title, append the movie info to the movie_info list
        if title != '':
            movie_info_dict = {'Title': title, 'Awards_Year': awards_year, 'Winner': winner}
            movie_info.append(movie_info_dict)
movie_info_df = pd.DataFrame(movie_info)
movie_info_df

,Title,Awards_Year,Winner
0,Wings,1928,Yes
1,7th Heaven,1928,No
2,The Racket,1928,No
3,The Broadway Melody,1929,Yes
4,Alibi,1929,No
...,...,...,...
606,Emilia Pérez,2024,No
607,I'm Still Here,2024,No
608,Nickel Boys,2024,No
609,The Substance,2024,No


In [14]:
movie_info_df.to_csv('best_picture.csv', index=True)

#### 2. Gather Movie Data via TMDB API  

##### 2a. Set up the API    
    * Create a free [TMDB account](https://developer.themoviedb.org/docs/getting-started)  
    * Generate an API key are review their documentation, especially:  
        * /discover/movie  
        * /movie/{movie_id}  
        * /search/movie  

In [17]:
with open("../tmdb_movies_key.json") as tmdb_movies:
    credentials = json.load(tmdb_movies)

api_key = credentials["key"]

In [18]:
endpoint = 'https://api.themoviedb.org/3/discover/movie'
movie_data = []
# Iterate through all years between 2015 and 2024
for year in range(2015,2025):
    # Each page contains 20 results, so we need to iterate through 5 pages to get 100 results
    for page in range(1,6):
        # Define params
        params = {
            'api_key': api_key,
            'primary_release_year': year,
            'sort_by': 'vote_count.desc',
            'page': page
        }
        # Get response
        response = requests.get(endpoint, params = params)
        res = response.json()['results']
        for movie in res:
            movie_data.append(movie)
        # Sleep before next API call
        time.sleep(0.25)

In [19]:
movie_data[0]

{'adult': False,
 'backdrop_path': '/kIBK5SKwgqIIuRKhhWrJn3XkbPq.jpg',
 'genre_ids': [28, 12, 878],
 'id': 99861,
 'original_language': 'en',
 'original_title': 'Avengers: Age of Ultron',
 'overview': 'When Tony Stark tries to jumpstart a dormant peacekeeping program, things go awry and Earth’s Mightiest Heroes are put to the ultimate test as the fate of the planet hangs in the balance. As the villainous Ultron emerges, it is up to The Avengers to stop him from enacting his terrible plans, and soon uneasy alliances and unexpected action pave the way for an epic and unique global adventure.',
 'popularity': 11.8411,
 'poster_path': '/4ssDuvEDkSArWEdyBl2X5EHvYKU.jpg',
 'release_date': '2015-04-22',
 'title': 'Avengers: Age of Ultron',
 'video': False,
 'vote_average': 7.271,
 'vote_count': 23847}

#####  2b. Collect top movies (2015-2024)  
    For each year from 2015 to 2024:  
        * Query TMDB for the top 100 movies (by vote count).  
        * For each movie, gather:  
            * Title  
            * Release Year  
            * Genre(s)  
            * Vote Average  
            * Vote Count  
            * Budget  
            * Revenue  
            * TMDB ID  
        * Store all results in a single DataFrame and export to movies_2015_2024.csv.
        * Hint: TMDB rate limits are generous for free accounts, but you should pause between requests (eg. time.sleep(0.25)). 
        * Some Oscar films may not appear in the top 100 by vote count. For any missing, use the /search/movie endpoint to add it.  

In [21]:
#mine = vote avg and vote count

In [22]:
movie_data[0]

{'adult': False,
 'backdrop_path': '/kIBK5SKwgqIIuRKhhWrJn3XkbPq.jpg',
 'genre_ids': [28, 12, 878],
 'id': 99861,
 'original_language': 'en',
 'original_title': 'Avengers: Age of Ultron',
 'overview': 'When Tony Stark tries to jumpstart a dormant peacekeeping program, things go awry and Earth’s Mightiest Heroes are put to the ultimate test as the fate of the planet hangs in the balance. As the villainous Ultron emerges, it is up to The Avengers to stop him from enacting his terrible plans, and soon uneasy alliances and unexpected action pave the way for an epic and unique global adventure.',
 'popularity': 11.8411,
 'poster_path': '/4ssDuvEDkSArWEdyBl2X5EHvYKU.jpg',
 'release_date': '2015-04-22',
 'title': 'Avengers: Age of Ultron',
 'video': False,
 'vote_average': 7.271,
 'vote_count': 23847}

In [36]:
for item in movie_data[0:5]:
    print(item['vote_average'])

7.271
7.627
7.91
6.7
7.69


In [58]:
vote_averages = [d['vote_average'] for d in movie_data]
vote_averages[:5]

[7.271, 7.627, 7.91, 6.7, 7.69]

In [60]:
movie_titles = [d['title'] for d in movie_data]
movie_titles[:10]

['Avengers: Age of Ultron',
 'Mad Max: Fury Road',
 'Inside Out',
 'Jurassic World',
 'The Martian',
 'Ant-Man',
 'Star Wars: The Force Awakens',
 'The Revenant',
 'Kingsman: The Secret Service',
 'The Hateful Eight']

In [64]:
vote_counts = [d['vote_count'] for d in movie_data]
vote_counts[:10]

[23847, 23503, 22917, 21094, 20579, 20416, 20067, 18777, 17383, 14806]

In [ ]:
vote_counts = [d['vote_count'] for d in movie_data]
vote_counts[:10]

In [42]:
movies_tmdb = pd.DataFrame(
    # naming the new columns and inputting the list data from collections
    {'vote_avg': vote_avg,
     'vote_count': vote_ct,
     })

In [46]:
movies_tmdb.head(2)

,vote_avg,vote_count
0,7.271,23847
1,7.627,23503


In [48]:
tmdb_ids = []
budgets = []
revenues = []
genre_names = []

for movie in movie_data:
    tmdb_ids.append(movie['id'])
for movie_id in tmdb_ids:
    # Use the movie ids to search for budget, revenue, and genre information
    endpoint = f'https://api.themoviedb.org/3/movie/{movie_id}'
    # Define params
    params = {
        'api_key': api_key,
    }
    # Get response
    response = requests.get(endpoint, params = params)
    res = response.json()
    # Extract budget, revenue, and genres
    budgets.append(res['budget'])
    revenues.append(res['revenue'])
    # Extract genre_ids and genres
    genre_names.append([genre['name'] for genre in res['genres']])
    # Sleep before next API call
    time.sleep(0.25)


In [49]:
len(tmdb_ids)

1000

In [88]:
release_year = []
for movie in movie_data:
    release_year.append(movie['release_date'])


In [90]:
years = [datetime.datetime.strptime(date_str, "%Y-%m-%d").year for date_str in release_year]

##### 2b.
        * Store all results in a single DataFrame and export to movies_2015_2024.csv.
        * Hint: TMDB rate limits are generous for free accounts, but you should pause between requests (eg. time.sleep(0.25)). 
        * Some Oscar films may not appear in the top 100 by vote count. For any missing, use the /search/movie endpoint to add it.  

In [103]:
movies_tmdb = pd.DataFrame(
    # naming the new columns and inputting the list data from collections
    {'Movie_Title': movie_titles, 
     'Release_Year': years, 
     'Genre': genre_names, 
     'Vote_Average': vote_averages, 
     'Vote_Count': vote_counts, 
     'Budget': budgets, 
     'Revenue': revenues, 
     'TMDB_ID': tmdb_ids
     })

In [109]:
movies_tmdb.head(2)

,Movie_Title,Release_Year,Genre,Vote_Average,Vote_Count,Budget,Revenue,TMDB_ID
0,Avengers: Age of Ultron,2015,"[Action, Adventure, Science Fiction]",7.271,23847,365000000,1405403694,99861
1,Mad Max: Fury Road,2015,"[Action, Adventure, Science Fiction]",7.627,23503,150000000,378858340,76341


In [107]:
movies_tmdb.to_csv('movie_2015_2024.csv', index=True)